# 成為資料分析師 | Python 與資料科學應用

> Pandas 101：處理表格式資料的 Python 套件

## 郭耀仁

## 大綱

- pandas 解決了什麼問題
- pandas 基礎
- Series 與 DataFrame 的基礎操作

## pandas 解決了什麼問題

## 請計算註冊於開曼群島（股票名稱有 KY 註記）的上市公司股價中位數

<https://tw.stock.yahoo.com/d/i/rank.php?t=pri&e=tse&n=100>

In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
def get_price_rank():
    query_str_parameters = {
        't': 'pri',
        'e': 'tse',
        'n': '100'
    }
    request_url = "https://tw.stock.yahoo.com/d/i/rank.php"
    response = requests.get(request_url, params=query_str_parameters)
    soup = BeautifulSoup(response.text)
    stock_tickers = []
    stock_names = []
    for e in soup.select(".name a"):
        stock_ticker = e.text.split()[0]
        stock_name = e.text.split()[1]
        stock_tickers.append(stock_ticker)
        stock_names.append(stock_name)
    prices = [float(e.text) for e in soup.select('.name+ td')]
    return stock_tickers, stock_names, prices

In [3]:
stock_tickers, stock_names, prices = get_price_rank()
print(stock_tickers)
print(stock_names)
print(prices)

['3008', '5269', '6415', '6409', '6669', '2454', '3661', '2207', '3406', '1590', '8454', '6531', '3533', '2330', '2379', '2327', '1476', '8464', '3653', '2059', '2049', '9921', '6504', '3443', '2395', '3563', '3034', '6414', '2227', '2912', '4968', '9914', '8341', '6230', '2357', '2474', '2345', '2404', '3413', '3665', '4137', '1256', '2308', '1325', '8070', '9910', '1707', '1477', '2492', '4551', '3023', '6491', '3515', '8462', '2383', '8081', '2439', '8422', '6670', '4536', '6582', '2360', '3130', '6706', '1537', '3376', '6416', '5215', '8016', '4958', '6533', '2458', '6271', '4439', '6715', '4766', '1558', '6269', '2707', '2377', '8499', '4438', '6278', '4977', '4912', '6456', '6213', '4763', '3532', '3044', '5434', '5871', '4943', '6464', '4119', '1760', '8480', '8046', '1232', '2231']
['大立光', '祥碩', '矽力-KY', '旭隼', '緯穎', '聯發科', '世芯-KY', '和泰車', '玉晶光', '亞德客-KY', '富邦媒', '愛普', '嘉澤', '台積電', '瑞昱', '國巨', '儒鴻', '億豐', '健策', '川湖', '上銀', '巨大', '南六', '創意', '研華', '牧德', '聯詠', '樺漢', '裕日車', '統一超', 

In [4]:
from statistics import median

ky_prices = [price for stock_name, price in zip(stock_names, prices) if "KY" in stock_name]
median(ky_prices)

139.0

## Python 一直以來都非常適合資料處理，但她的分析能力很薄弱，`pandas` 的開發有助於補足 Python 資料分析的需求，讓使用者能夠在 Python 中執行完整的資料分析流程，而無需切換到 data-centric 的特定語言，如 R。

In [5]:
import pandas as pd

df = pd.DataFrame()
df["ticker"] = stock_tickers
df["stock_name"] = stock_names
df["price"] = prices
df.head()

,ticker,stock_name,price
0,3008,大立光,4000.0
1,5269,祥碩,1815.0
2,6415,矽力-KY,1785.0
3,6409,旭隼,980.0
4,6669,緯穎,838.0


In [6]:
df[df["stock_name"].str.contains("KY")]["price"].median()

139.0

## pandas 提供了新的資料結構

- Series 是具備索引的 ndarray
- DataFrame 能完美處理表格式資料（tabular data）
- Panel 能為 DataFrame 加入第三個維度，通常為時間（自 Pandas 0.20.0 版本之後取消了此類別）

## pandas 主要的應用場景

- 表格式資料的讀取
- 豐富的資料清理與分析函數
- 視覺化：包裝了常用的 matplotlib.pyplot 圖形

## pandas 基礎

## 什麼是 pandas？

> Flexible and powerful data analysis / manipulation library for Python, providing labeled data structures similar to R data.frame objects, statistical functions, and much more.

## 安裝與載入 pandas

- 安裝

```bash
# run in bash shell
pip install pandas
```

- 載入

```py
# run in python console
import pandas as pd
```

## pandas 的命名源自她的資料結構

- **Pan**el（自 Pandas 0.20.0 版本之後取消了此類別）
- **Da**taFrame
- **S**eries
- Index

## pandas 中的 Series

使用 `pd.Series()` 函數創建 Series 類別，Series 從 ndarray 繼承了所有特性，並加上一組 Index。

In [7]:
import pandas as pd

movie_ratings = [8.0, 7.3, 8.5, 8.6]
ser = pd.Series(movie_ratings)
print(type(ser))
print(ser)
print(ser[3])

<class 'pandas.core.series.Series'>
0    8.0
1    7.3
2    8.5
3    8.6
dtype: float64
8.6


## 這使得她不只能夠透過絕對位置來索引，亦可以透過像操作 dict 一般，以鍵（Key）作為選擇索引依據

In [8]:
movie_ratings = [8.0, 7.3, 8.5, 8.6]
movie_titles = ["The Avengers", "Avengers: Age of Ultron", "Avengers: Infinity War", "Avengers: Endgame"]
ser = pd.Series(movie_ratings, index=movie_titles)
print(ser)
print(ser["Avengers: Endgame"])

The Avengers               8.0
Avengers: Age of Ultron    7.3
Avengers: Infinity War     8.5
Avengers: Endgame          8.6
dtype: float64
8.6


## 可以將 Series 視為一種較為泛用的 ndarray，同時具備 list 和 dict 的特性，以 .index 屬性與 .values 屬性可以將 Series 拆分為 Index 類別與 ndarray

In [9]:
movie_ratings = [8.0, 7.3, 8.5, 8.6]
movie_titles = ["The Avengers", "Avengers: Age of Ultron", "Avengers: Infinity War", "Avengers: Endgame"]
ser = pd.Series(movie_ratings, index=movie_titles)
print(ser.index)
print(ser.values)
print(type(ser.index))
print(type(ser.values))

Index(['The Avengers', 'Avengers: Age of Ultron', 'Avengers: Infinity War',
       'Avengers: Endgame'],
      dtype='object')
[8.  7.3 8.5 8.6]
<class 'pandas.core.indexes.base.Index'>
<class 'numpy.ndarray'>


## pandas 中的 DataFrame

我們可以使用 `pd.DataFrame()` 函數創建 DataFrame 類別。

In [10]:
movie_ratings = [8.0, 7.3, 8.5, 8.6]
movie_titles = ["The Avengers", "Avengers: Age of Ultron", "Avengers: Infinity War", "Avengers: Endgame"]
df = pd.DataFrame()
df["title"] = movie_titles
df["rating"] = movie_ratings
print(type(df))
df

<class 'pandas.core.frame.DataFrame'>


,title,rating
0,The Avengers,8.0
1,Avengers: Age of Ultron,7.3
2,Avengers: Infinity War,8.5
3,Avengers: Endgame,8.6


## DataFrame 將多組共享 Index 的 Series 組合為一個具備列索引（row index）與欄標籤（column label）的資料集，我們可以進一步分拆成列索引、欄標籤與 Series

In [11]:
movie_ratings = [8.0, 7.3, 8.5, 8.6]
movie_titles = ["The Avengers", "Avengers: Age of Ultron", "Avengers: Infinity War", "Avengers: Endgame"]
df = pd.DataFrame()
df["title"] = movie_titles
df["rating"] = movie_ratings
print(df.index)
print(df.columns)
print(df["title"])
print(df["rating"])

RangeIndex(start=0, stop=4, step=1)
Index(['title', 'rating'], dtype='object')
0               The Avengers
1    Avengers: Age of Ultron
2     Avengers: Infinity War
3          Avengers: Endgame
Name: title, dtype: object
0    8.0
1    7.3
2    8.5
3    8.6
Name: rating, dtype: float64


## pandas 中的 Index

不論是 Series 或 DataFrame 物件都包含一個 Index 類別，作為萃取以及更新資料的根據，Index 可以被視為是一種結合了 tuple 的不可變（Immutable）特性以及 set 集合運算特性的資料結構類別，我們可以使用 `pd.Index()` 函數創建出下列的範例

In [12]:
pd_index = pd.Index([0, 2, 3, 4])
print(type(pd_index))
print(pd_index)
pd_index[0] = 1

<class 'pandas.core.indexes.numeric.Int64Index'>
Int64Index([0, 2, 3, 4], dtype='int64')


TypeError: Index does not support mutable operations

## 創建後不能更新，Index 也支援 Set 類別的集合運算，可以對兩組 Index 類別（如例子中的五個奇數、四個質數）使用交集、聯集與 XOR（Exclusive OR）

In [13]:
odds_index = pd.Index([1, 3, 5, 7, 9])
primes_index = pd.Index([2, 3, 5, 7])
print(odds_index & primes_index) # and
print(odds_index | primes_index) # or
print(odds_index ^ primes_index) # exclusive or

Int64Index([3, 5, 7], dtype='int64')
Int64Index([1, 2, 3, 5, 7, 9], dtype='int64')
Int64Index([1, 2, 9], dtype='int64')


## Series 與 DataFrame 的基礎操作

## 基礎操作包含

- 創建
- 資料載入
- 索引、切割與篩選
- 排序
- 衍生變數
- 摘要、分組摘要

## 常見創建 Series 的方式是使用 `pd.Series()` 函數傳入一個「類似清單（list-like）」的物件，包含 list 、tuple 或 ndarray，如果沒有指定另外一個「類似清單」的物件作為索引，Series 會自動以類似 `range()` 函數設定對應長度的索引

In [14]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
ser = pd.Series(movie_ratings)
print(ser)

0    9.0
1    8.9
2    8.8
3    8.7
dtype: float64


## 在 `pd.Series()` 函數中加入 index 參數指定對應長度的索引

In [15]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
ser = pd.Series(movie_ratings, index=movie_titles)
print(ser.index)
print(ser.values)
print(ser)

Index(['The Dark Knight', 'Schindler's List', 'Forrest Gump', 'Inception'], dtype='object')
[9.  8.9 8.8 8.7]
The Dark Knight     9.0
Schindler's List    8.9
Forrest Gump        8.8
Inception           8.7
dtype: float64


## Series 被設計成由一組索引與一組資料所搭建而成的資料結構，因此我們亦可以在 `pd.Series()` 函數中傳入 dict，如此一來字典中的鍵（Keys）會被記錄成為索引、字典中的值（Values）則會被記錄成為陣列中的資料

In [16]:
movie_dict = {
    "The Dark Knight": 9.0,
    "Schindler's List": 8.9,
    "Forrest Gump": 8.8,
    "Inception": 8.7
}
ser = pd.Series(movie_dict)
print(movie_dict.keys())
print(movie_dict.values())
print("\n")
print(ser.index)
print(ser.values)
print(ser)

dict_keys(['The Dark Knight', "Schindler's List", 'Forrest Gump', 'Inception'])
dict_values([9.0, 8.9, 8.8, 8.7])


Index(['The Dark Knight', 'Schindler's List', 'Forrest Gump', 'Inception'], dtype='object')
[9.  8.9 8.8 8.7]
The Dark Knight     9.0
Schindler's List    8.9
Forrest Gump        8.8
Inception           8.7
dtype: float64


## 常見創建 DataFrame 的方式是使用 `pd.DataFrame()` 函數傳入多個「類似清單（list-like）」的物件，包含 list 、tuple 或 ndarray，並視需求指定變數名稱或索引；同樣地，若是在沒有指定索引的情況下，DataFrame 會自動以類似 `range()` 函數設定對應長度的索引

In [17]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
df = pd.DataFrame()
df["title"] = movie_titles
df["rating"] = movie_ratings
df

,title,rating
0,The Dark Knight,9.0
1,Schindler's List,8.9
2,Forrest Gump,8.8
3,Inception,8.7


## 與 Series 相同，DataFrame 被設計成由一組索引與多組類似清單資料所搭建而成的資料結構，因此我們亦可以在 `pd.DataFrame()` 函數中傳入 dict，如此一來字典中的鍵（Keys）會被記錄成為變數名稱、字典中的值（Values）則會被記錄成為陣列中的資料

In [18]:
movie_dict = {
    "title": ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"],
    "rating": [9.0, 8.9, 8.8, 8.7]
}
df = pd.DataFrame(movie_dict)
df

,title,rating
0,The Dark Knight,9.0
1,Schindler's List,8.9
2,Forrest Gump,8.8
3,Inception,8.7


## 實務應用資料框物件時，多數情況都不會是手動輸入資料內容，而是由外部資料源載入

- 以逗號區隔變數的 CSV 文字檔
- 以 JSON 所組成的陣列文字檔
- 試算表
- 資料庫中的表格

## 使用 `pd.read_csv()` 函數讀入以逗號區隔變數的 CSV 文字檔

In [19]:
df = pd.read_csv("https://python4ds.s3-ap-northeast-1.amazonaws.com/movies.csv")
df

,title,rating
0,The Dark Knight,9.0
1,Schindler's List,8.9
2,Forrest Gump,8.8
3,Inception,8.7


## 使用 `pd.read_json()` 函數讀入以 JSON 所組成的陣列文字檔

In [20]:
df = pd.read_json("https://python4ds.s3-ap-northeast-1.amazonaws.com/movies.json")
df

,title,rating
0,The Dark Knight,9.0
1,Schindler's List,8.9
2,Forrest Gump,8.8
3,Inception,8.7


## 使用 `pd.read_excel()` 函數讀入試算表

In [21]:
df = pd.read_excel("https://python4ds.s3-ap-northeast-1.amazonaws.com/movies.xlsx")
df

,title,rating
0,The Dark Knight,9.0
1,Schindler's List,8.9
2,Forrest Gump,8.8
3,Inception,8.7


## 使用 `pd.read_sql()` 函數讀入資料庫中的表格

In [22]:
import sqlite3

# Creating a demo.db database in working directory
conn = sqlite3.connect('demo.db')
# Importing a table
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
df = pd.DataFrame()
df["title"] = movie_titles
df["rating"] = movie_ratings
df.to_sql("movies", index=False, con=conn, if_exists='replace')
# Importing data from demo.movies
query_str = """
SELECT *
    FROM movies
    WHERE rating < 9.0;
"""
pd.read_sql(query_str, con=conn)

,title,rating
0,Schindler's List,8.9
1,Forrest Gump,8.8
2,Inception,8.7


## Series 的索引、切割與篩選

In [23]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
ser = pd.Series(movie_ratings, index=movie_titles)
print(ser)
print(ser[0])
print(ser["Forrest Gump"])

The Dark Knight     9.0
Schindler's List    8.9
Forrest Gump        8.8
Inception           8.7
dtype: float64
9.0
8.8


## 進行資料值的切割時，可以在中括號輸入所需資料的起點與終點，傳入絕對位置則與 Python 慣例一致：不包含終點；若是傳入索引值則會包含終點

In [24]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
ser = pd.Series(movie_ratings, index=movie_titles)
print(ser[1:4])
print(ser["Schindler's List":"Inception"])

Schindler's List    8.9
Forrest Gump        8.8
Inception           8.7
dtype: float64
Schindler's List    8.9
Forrest Gump        8.8
Inception           8.7
dtype: float64


## Series 繼承了 ndarray 的所有特性，因此完全適用華麗索引與布林索引這兩個便利的技法

In [25]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
ser = pd.Series(movie_ratings, index=movie_titles)
# Fancy indexing
print(ser[[1, 2, 3]])
print(ser[["Schindler's List", "Forrest Gump", "Inception"]])
# Boolean indexing
print(ser < 9)
print(ser[ser < 9])

Schindler's List    8.9
Forrest Gump        8.8
Inception           8.7
dtype: float64
Schindler's List    8.9
Forrest Gump        8.8
Inception           8.7
dtype: float64
The Dark Knight     False
Schindler's List     True
Forrest Gump         True
Inception            True
dtype: bool
Schindler's List    8.9
Forrest Gump        8.8
Inception           8.7
dtype: float64


## DataFrame 的選擇與篩選

利用 `[COLUMN]` 或 `.COLUMN` 能夠從資料框中選擇出單一或多個變數，成為一個 Series 或者變數欄位較少的資料框子集，實踐 SQL 語法中的 SELECT

In [26]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
release_years = [2008, 1993, 1994, 2010]
df = pd.DataFrame()
df["title"] = movie_titles
df["rating"] = movie_ratings
df["release_year"] = release_years
print(df["title"])
print(df.rating)
df[["title", "release_year"]]

0     The Dark Knight
1    Schindler's List
2        Forrest Gump
3           Inception
Name: title, dtype: object
0    9.0
1    8.9
2    8.8
3    8.7
Name: rating, dtype: float64


,title,release_year
0,The Dark Knight,2008
1,Schindler's List,1993
2,Forrest Gump,1994
3,Inception,2010


## 對資料框直接應用布林索引可以挑出符合條件（條件判斷結果為 True）的觀測值列數，實踐 SQL 語法中的 WHERE

In [27]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
release_years = [2008, 1993, 1994, 2010]
df = pd.DataFrame()
df["title"] = movie_titles
df["rating"] = movie_ratings
df["release_year"] = release_years
print(df["release_year"] > 2000)
df[df["release_year"] > 2000]

0     True
1    False
2    False
3     True
Name: release_year, dtype: bool


,title,rating,release_year
0,The Dark Knight,9.0,2008
3,Inception,8.7,2010


## 資料框類別尚有兩個索引語法（Indexer）可以方便使用者在一次的函數呼叫中就完成選擇與篩選兩個任務

- `.loc[ROW_LABEL, COLUMN_LABEL]` ：純粹以列、欄標籤為準進行選擇跟篩選
- `.iloc[ROW_INDEX, COLUMN_INDEX]` ：純粹以資料的整數位置（integer location）為準進行選擇跟篩選

In [28]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
release_years = [2008, 1993, 1994, 2010]
df = pd.DataFrame()
df["rating"] = movie_ratings
df["release_year"] = release_years
df.index = movie_titles
df

,rating,release_year
The Dark Knight,9.0,2008
Schindler's List,8.9,1993
Forrest Gump,8.8,1994
Inception,8.7,2010


## 希望將上映年份在 2000 年之後的 “The Dark Knight” 與 “Inception” 利用索引語法選出來

使用 `.loc[]` 時必須傳入列標籤（電影名稱）與欄標籤（評等和上映年份）

In [29]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
release_years = [2008, 1993, 1994, 2010]
df = pd.DataFrame()
df["rating"] = movie_ratings
df["release_year"] = release_years
df.index = movie_titles
df.loc[["The Dark Knight", "Inception"], ["rating", "release_year"]]

,rating,release_year
The Dark Knight,9.0,2008
Inception,8.7,2010


## 希望將上映年份在 2000 年之後的 “The Dark Knight” 與 “Inception” 利用索引語法選出來

使用 `.iloc[]` 時必須傳入 “The Dark Knight” 與 “Inception” 的列整數位置（第 0 與 3 列）與欄整數位置（第 0 與 1 欄）

In [30]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
release_years = [2008, 1993, 1994, 2010]
df = pd.DataFrame()
df["rating"] = movie_ratings
df["release_year"] = release_years
df.index = movie_titles
df.iloc[[0, 3], [0, 1]]

,rating,release_year
The Dark Knight,9.0,2008
Inception,8.7,2010


## 排序

- `df.sort_index()` ：依照資料框的列標籤遞增（預設）或遞減排序
- `df.sort_values()` ：依照指定的資料框欄標籤遞增（預設）或遞減排序

In [31]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
release_years = [2008, 1993, 1994, 2010]
df = pd.DataFrame()
df["title"] = movie_titles
df["rating"] = movie_ratings
df["release_year"] = release_years
df.sort_index() # default: ascending

,title,rating,release_year
0,The Dark Knight,9.0,2008
1,Schindler's List,8.9,1993
2,Forrest Gump,8.8,1994
3,Inception,8.7,2010


In [32]:
df.sort_index(ascending=False)

,title,rating,release_year
3,Inception,8.7,2010
2,Forrest Gump,8.8,1994
1,Schindler's List,8.9,1993
0,The Dark Knight,9.0,2008


In [33]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
release_years = [2008, 1993, 1994, 2010]
df = pd.DataFrame()
df["title"] = movie_titles
df["rating"] = movie_ratings
df["release_year"] = release_years
df.sort_values("title")

,title,rating,release_year
2,Forrest Gump,8.8,1994
3,Inception,8.7,2010
1,Schindler's List,8.9,1993
0,The Dark Knight,9.0,2008


In [34]:
df.sort_values("release_year")

,title,rating,release_year
1,Schindler's List,8.9,1993
2,Forrest Gump,8.8,1994
0,The Dark Knight,9.0,2008
3,Inception,8.7,2010


In [35]:
df.sort_values("release_year", ascending=False)

,title,rating,release_year
3,Inception,8.7,2010
0,The Dark Knight,9.0,2008
2,Forrest Gump,8.8,1994
1,Schindler's List,8.9,1993


## 衍生變數

- 簡單運算
- 類別對應類別
- 數值對應類別
- 函數映射

## 簡單運算

透過 Series 從 ndarray 繼承而來的元素級別運算（element-wise operation）特性即可實踐，像是運用身高、體重這兩個欄位衍生出球員的 BMI

In [36]:
player_profile = pd.read_csv("https://python4ds.s3-ap-northeast-1.amazonaws.com/player_profile.csv")
player_profile["bmi"] = player_profile["weightKilograms"] / player_profile["heightMeters"]**2
player_profile[["temporaryDisplayName", "bmi"]].head()

,temporaryDisplayName,bmi
0,"Adams, Steven",26.493861
1,"Adebayo, Bam",26.742788
2,"Adel, Deng",22.449939
3,"Aldridge, LaMarcus",26.481885
4,"Alexander, Kyle",22.416388


## 類別對應類別

透過 Series 的 `.map()` 方法來實踐，傳入 dict 作為對應的準則，字典的鍵（Key）為對應前的原始類別，字典的值（Value）為對應後的類別，例如將本來分類較細膩的鋒衛對應為較粗略的前場、後場

In [37]:
pos_dict = {
    "G": "Backcourt",
    "F": "Frontcourt",
    "C": "Frontcourt",
    "G-F": "Backcourt",
    "F-C": "Frontcourt",
    "F-G": "Frontcourt",
    "C-F": "Frontcourt"
}
player_profile = pd.read_csv("https://python4ds.s3-ap-northeast-1.amazonaws.com/player_profile.csv")
print("Pos before mapping:")
player_profile["pos"].value_counts()

Pos before mapping:


G      200
F      186
C       51
G-F     39
F-C     20
F-G     15
C-F     13
Name: pos, dtype: int64

In [38]:
print("Pos after mapping:")
player_profile["pos_recoded"] = player_profile["pos"].map(pos_dict)
player_profile["pos_recoded"].value_counts()

Pos after mapping:


Frontcourt    285
Backcourt     239
Name: pos_recoded, dtype: int64

## 數值對應類別

透過 `pd.cut()` 函數將數值變數依照指定的門檻值或箱數切分成為類別變數，舉例來說將身高對應為小於等於 2 公尺以及超過 2 公尺兩個類別

In [39]:
import numpy as np

player_profile = pd.read_csv("https://python4ds.s3-ap-northeast-1.amazonaws.com/player_profile.csv")
player_profile["heightCategory"] = pd.cut(player_profile["heightMeters"], [0, 2, np.Inf], labels=["<= 2m", "> 2m"])
player_profile[["temporaryDisplayName", "heightMeters", "heightCategory"]].head(10)

,temporaryDisplayName,heightMeters,heightCategory
0,"Adams, Steven",2.13,> 2m
1,"Adebayo, Bam",2.08,> 2m
2,"Adel, Deng",2.01,> 2m
3,"Aldridge, LaMarcus",2.11,> 2m
4,"Alexander, Kyle",2.11,> 2m
5,"Alexander-Walker, Nickeil",1.96,<= 2m
6,"Allen, Grayson",1.96,<= 2m
7,"Allen, Jarrett",2.11,> 2m
8,"Allen, Kadeem",1.90,<= 2m
9,"Aminu, Al-Farouq",2.06,> 2m


## 函數映射

透過 `.apply()` 方法來實踐，傳入函數或 lambda 表示式作為映射的準則，例如將本來分類較細膩的鋒衛對應為較粗略的 G、F 與 C

In [40]:
def recode_pos(x):
    if x[0] == 'G':
        return 'G'
    elif x[0] == 'F':
        return 'F'
    elif x[0] == 'C':
        return 'C'

player_profile = pd.read_csv("https://python4ds.s3-ap-northeast-1.amazonaws.com/player_profile.csv")
player_profile["pos_recoded"] = player_profile["pos"].apply(recode_pos)
print("Pos before applying:")
player_profile["pos"].value_counts()

Pos before applying:


G      200
F      186
C       51
G-F     39
F-C     20
F-G     15
C-F     13
Name: pos, dtype: int64

In [41]:
print("Pos after applying:")
player_profile["pos_recoded"].value_counts()

Pos after applying:


G    239
F    221
C     64
Name: pos_recoded, dtype: int64

## 摘要

對資料框呼叫常用的摘要方法

- `.count()` 計算列數
- `.mean()` 與 `.median()` 計算平均和中位數
- `.min()` 與 `.max()` 計算最小和最大值
- `.std()` 與 `.var()` 計算標準差和變異數
- `.prod()` 計算乘積
- `.sum()` 計算總和

## 摘要方法預設作用的維度是資料框的欄位，比方從上映年份、評等與片長三個變數中取出各自的最大值

In [42]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
release_years = [2008, 1993, 1994, 2010]
movie_length_mins = [152, 195, 142, 148]
df = pd.DataFrame()
df["rating"] = movie_ratings
df["release_year"] = release_years
df["movie_length_mins"] = movie_length_mins
df.index = movie_titles
print(df.max())
print("\n")
print(df.max(axis=1))

rating                  9.0
release_year         2010.0
movie_length_mins     195.0
dtype: float64


The Dark Knight     2008.0
Schindler's List    1993.0
Forrest Gump        1994.0
Inception           2010.0
dtype: float64


## 獲取資料的最大值最小值

- `df["col_name"].max()`
- `df["col_name"].min()`

## 獲取資料最大最小的索引值

- `df["col_name"].idxmax()`
- `df["col_name"].idxmin()`

## 獲取類別資料的摘要

- `df["col_name"].nunique()`
- `df["col_name"].unique()`
- `df["col_name"].value_counts()`

In [43]:
player_profile = pd.read_csv("https://python4ds.s3-ap-northeast-1.amazonaws.com/player_profile.csv")
print(player_profile["country"].size)
print(player_profile["country"].nunique())
print(player_profile["country"].unique())

524
42
['New Zealand' 'USA' 'South Sudan' 'Canada' 'Greece' 'United Kingdom'
 'Bahamas' 'France' 'Australia' 'Italy' 'Croatia' 'Latvia' 'Georgia'
 'Democratic Republic of the Congo' 'Serbia' 'Sudan' 'Germany' 'Lithuania'
 'Brazil' 'Slovenia' 'Switzerland' nan 'Mali' 'Senegal' 'Guinea'
 'Cameroon' 'Angola' ' ' 'Spain' 'Yugoslavia' 'Japan' 'Dominican Republic'
 'Turkey' 'Haiti' 'Ukraine' 'Finland' 'Bosnia and Herzegovina' 'Egypt'
 'Nigeria' 'Austria' 'Czech Republic' 'Gabon' 'Montenegro']


In [44]:
player_profile["country"].value_counts()

USA                                 393
Canada                               20
France                               12
Australia                            10
Croatia                               7
Serbia                                6
Germany                               6
Spain                                 5
Lithuania                             4
Turkey                                4
Latvia                                3
Slovenia                              3
Greece                                3
Italy                                 3
Brazil                                3
Democratic Republic of the Congo      3
Senegal                               2
Cameroon                              2
Japan                                 2
Mali                                  2
Bosnia and Herzegovina                2
United Kingdom                        2
Bahamas                               2
Sudan                                 2
South Sudan                           2


## 分組摘要

更多的應用情境中我們會指定一些類別變數分組，在分組的前提下對資料框變數呼叫簡單的摘要方法，這樣的操作源自於 SQL 資料庫查詢語言的 GROUP BY 語法

In [45]:
player_profile = pd.read_csv("https://python4ds.s3-ap-northeast-1.amazonaws.com/player_profile.csv")
groupby_object = player_profile.groupby("pos")
print(groupby_object["heightMeters"].mean()) # Average height by pos
print("\n")
print(groupby_object["weightKilograms"].mean()) # Average weight by pos

pos
C      2.126863
C-F    2.090000
F      2.044378
F-C    2.107500
F-G    1.995333
G      1.924800
G-F    2.000000
Name: heightMeters, dtype: float64


pos
C      114.335294
C-F    110.676923
F      101.589730
F-C    110.855000
F-G     95.126667
G       88.896000
G-F     95.717949
Name: weightKilograms, dtype: float64


## 常用的列索引相關方法

- `df.set_index("col_name")`
- `df.reset_index()`

In [46]:
movie_ratings = [9.0, 8.9, 8.8, 8.7]
movie_titles = ["The Dark Knight", "Schindler's List", "Forrest Gump", "Inception"]
release_years = [2008, 1993, 1994, 2010]
df = pd.DataFrame()
df["title"] = movie_titles
df["rating"] = movie_ratings
df["release_year"] = release_years
df

,title,rating,release_year
0,The Dark Knight,9.0,2008
1,Schindler's List,8.9,1993
2,Forrest Gump,8.8,1994
3,Inception,8.7,2010


In [47]:
df = df.set_index("title")
df

,rating,release_year
title,,
The Dark Knight,9.0,2008
Schindler's List,8.9,1993
Forrest Gump,8.8,1994
Inception,8.7,2010


In [48]:
df = df.reset_index()
df

,title,rating,release_year
0,The Dark Knight,9.0,2008
1,Schindler's List,8.9,1993
2,Forrest Gump,8.8,1994
3,Inception,8.7,2010


## 延伸閱讀

[pandas: powerful Python data analysis toolkit](http://pandas.pydata.org/pandas-docs/stable/)

## 隨堂練習

[隨堂練習：奧運獎牌排行](https://mybinder.org/v2/gh/yaojenkuo/python-data-analysis/master?filepath=exercises%2F03-exercises.ipynb)

## 題庫來源

[Introduction to Data Science in Python](https://www.coursera.org/learn/python-data-analysis)

In [49]:
request_url = "https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table"
html_tables = pd.read_html(request_url)
olympic_medal_table = html_tables[1]
column_names = ['team', 'no_summer_games', 'no_summer_golds', 'no_summer_silvers', 'no_summer_bronzes', 'no_summer_totals',
               'no_winter_games', 'no_winter_golds', 'no_winter_silvers', 'no_winter_bronzes', 'no_winter_totals',
               'no_combined_games', 'no_combined_golds', 'no_combined_silvers', 'no_combined_bronzes', 'no_combined_totals']
olympic_medal_table.columns = column_names
team_name_split = olympic_medal_table['team'].str.split('\(|\)|\[|\]', expand=True)
team_names = team_name_split[0].str.strip()
team_ioc = team_name_split[1].str.strip()
olympic_medal_table.insert(0, 'team_name', team_names.values)
olympic_medal_table.insert(1, 'team_ioc', team_ioc.values)
olympic_medal_table = olympic_medal_table.drop('team', axis=1)

In [50]:
olympic_medal_table.head()

,team_name,team_ioc,no_summer_games,no_summer_golds,no_summer_silvers,no_summer_bronzes,no_summer_totals,no_winter_games,no_winter_golds,no_winter_silvers,no_winter_bronzes,no_winter_totals,no_combined_games,no_combined_golds,no_combined_silvers,no_combined_bronzes,no_combined_totals
0,Afghanistan,AFG,14,0,0,2,2,0,0,0,0,0,14,0,0,2,2
1,Algeria,ALG,13,5,4,8,17,3,0,0,0,0,16,5,4,8,17
2,Argentina,ARG,24,21,25,28,74,19,0,0,0,0,43,21,25,28,74
3,Armenia,ARM,6,2,6,6,14,7,0,0,0,0,13,2,6,6,14
4,Australasia,ANZ,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


In [51]:
olympic_medal_table.tail()

,team_name,team_ioc,no_summer_games,no_summer_golds,no_summer_silvers,no_summer_bronzes,no_summer_totals,no_winter_games,no_winter_golds,no_winter_silvers,no_winter_bronzes,no_winter_totals,no_combined_games,no_combined_golds,no_combined_silvers,no_combined_bronzes,no_combined_totals
148,Zimbabwe,ZIM,13,3,4,1,8,1,0,0,0,0,14,3,4,1,8
149,Independent Olympic Athletes,IOA,3,1,0,1,2,0,0,0,0,0,3,1,0,1,2
150,Independent Olympic Participants,IOP,1,0,1,2,3,0,0,0,0,0,1,0,1,2,3
151,Mixed team,ZZX,3,8,5,4,17,0,0,0,0,0,3,8,5,4,17
152,Totals,None,28,5116,5081,5488,15685,23,1062,1058,1050,3170,51,6178,6139,6538,18855


## 隨堂練習：哪個國家贏得的夏季奧運金牌數最多？

In [52]:
def most_summer_gold_country(olympic_medal_table):
    """
    >>> most_summer_gold_country(olympic_medal_table)
    'United States'
    """

## 隨堂練習：哪個國家夏季奧運與冬季奧運的金牌數差距數最大？

In [53]:
def largest_gold_difference(olympic_medal_table):
    """
    >>> largest_gold_difference(olympic_medal_table)
    'United States'
    """

## 隨堂練習：哪個國家夏季奧運與冬季奧運的金牌數差距除以總金牌數的比例最大？（僅考慮至少有一個夏季金牌與一個冬季金牌的國家）

$$\text{Ratio} = \frac{\text{Summer Gold} - \text{Winter Gold}}{\text{Total Gold}}$$

In [54]:
def largest_ratio(olympic_medal_table):
    """
    >>> largest_ratio(olympic_medal_table)
    'Hungary'
    """

## 隨堂練習參考解答

[隨堂練習：奧運獎牌排行參考解答](https://mybinder.org/v2/gh/yaojenkuo/python-data-analysis/master?filepath=suggested_answers%2F03-suggested-answers.ipynb)